In [37]:
# change the current dir so the sys know the .py modules
import os
os.chdir('/home/patrick/eye-tracking/python')

FileNotFoundError: [Errno 2] No such file or directory: '/home/patrick/eye-tracking/python'

In [4]:
import ijson
import base64
import cv2
import random
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from cnn_utils import *
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

## Get data from the DB

### Verify if the actual data in the df has disordered ids compared to our API

Conclusion: No data in the db has difference from the mathed id in our API

In [ ]:
df = create_df_with_ids()

In [6]:
verify_df(df)

[]

In [44]:
df = create_df_id

NameError: name 'create_df_id' is not defined

### Verify if the matched id of x_train and y_train are different when we spliting them apart
Conclusion: we did not screw this step up

In [22]:
x_train, x_validation, y_train, y_validation = create_train_validation(df)
y_train_quaternary = create_quaternary_labels(y_train)
y_validation_quaternary = create_quaternary_labels(y_validation) # generate binary y labels

In [44]:
mask = df['id'].isin(list(x_train["id"]))
mask

0       True
1       True
2      False
3       True
4       True
       ...  
841     True
842    False
843     True
844     True
845     True
Name: id, Length: 846, dtype: bool

In [49]:
(df.loc[mask]["y"]==y_train["y"]).all()

True

In [50]:
y_train_quaternary = create_quaternary_labels(y_train)
y_validation_quaternary = create_quaternary_labels(y_validation) # generate binary y labels


In [51]:
y_validation_quaternary

,y
2,UR
6,LR
9,LR
11,LR
24,UL
...,...
824,UL
825,UR
826,UR
828,UL


In [62]:
quaternary_encoder = LabelEncoder()
quaternary_encoder.fit(y_train_quaternary)
quaternary_train = quaternary_encoder.transform(y_train_quaternary)
quaternary_test = quaternary_encoder.transform(y_validation_quaternary)

# one-hot encoding the data
quaternary_train = tf.keras.utils.to_categorical(quaternary_train)
quaternary_test = tf.keras.utils.to_categorical(quaternary_test)

## Simple CNN

In [21]:
IMG_WIDTH = x_train["eyeImage"].iloc[0].shape[1]
IMG_HEIGHT = x_train["eyeImage"].iloc[0].shape[0]

### Compile the simple CNN

In [23]:
tf.convert_to_tensor(x_train["eyeImage"].iloc[0])

<tf.Tensor 'Const:0' shape=(224, 224, 3) dtype=float64>

In [24]:
# target = df.pop('y')

# dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
data = {'X1':  [[1,2,3], [3,4,5]],
        'X2': [['a','b','c'],['c','d','e']]}
df = pd.DataFrame(data)
np.array(df)[1]

array([list([3, 4, 5]), list(['c', 'd', 'e'])], dtype=object)

In [25]:
def create_tf_data(X, Y):
    """
    take in the X and Y and transform each column into np array
    """
    
    eyeImage = np.stack(X['eyeImage'].to_numpy())
    leftEye = np.stack(X['leftEye'].to_numpy())
    rightEye = np.stack(X['rightEye'].to_numpy())
    y = np.stack(Y['y'].to_numpy())
    return eyeImage, leftEye, rightEye, y

In [26]:
eyeImage_train, leftEye_train, rightEye_train, binary_train = create_tf_data(x_train, y_train_binary)
eyeImage_test, leftEye_test, rightEye_test, binary_test = create_tf_data(x_validation, y_validation_binary)

NameError: name 'y_train_binary' is not defined

In [27]:
# encode the y label to make it suitable for CNN
binary_encoder = LabelEncoder()
binary_encoder.fit(binary_train)
binary_train = binary_encoder.transform(binary_train)
binary_test = binary_encoder.transform(binary_test)

NameError: name 'binary_train' is not defined

In [28]:
eyeImage = tf.keras.layers.Input(shape=(IMG_HEIGHT,IMG_WIDTH, 3))
leftEye = tf.keras.layers.Input(shape=(12)) # requires the eye positions to be flattened to 1D
rightEye = tf.keras.layers.Input(shape=(12))

conv = tf.keras.layers.Conv2D(filters=20, kernel_size=5, strides=1, activation='relu', kernel_initializer='VarianceScaling')(eyeImage)
maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2))(conv)
flat = tf.keras.layers.Flatten()(maxpool)
drop = tf.keras.layers.Dropout(rate=0.2)(flat)
concat = tf.keras.layers.concatenate([drop, leftEye, rightEye])
dense = tf.keras.layers.Dense(units=128,activation="relu",kernel_initializer='VarianceScaling')(concat)
out = tf.keras.layers.Dense(units=1,activation="sigmoid")(dense)
simple_model = tf.keras.Model(inputs=[eyeImage, leftEye, rightEye], outputs=out, name="SimpleModel")


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [29]:
simple_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'],loss = tf.keras.losses.MeanSquaredError())

simple_model.summary()

Model: "SimpleModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 220, 220, 20) 1520        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 110, 110, 20) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
flatten (Flatten)               (None, 242000)       0           max_pooling2d[0][0]              
________________________________________________________________________________________

In [30]:
simple_model.fit([eyeImage_train, leftEye_train, rightEye_train], binary_train, batch_size=64, epochs=100)

NameError: name 'eyeImage_train' is not defined

In [31]:
test_loss, test_acc = simple_model.evaluate([eyeImage_test, leftEye_test, rightEye_test], binary_test)
print('Test accuracy:', test_acc)

NameError: name 'eyeImage_test' is not defined